"""<br>
    @Author: Prayag Bhoir<br>
    @Date: 10-09-2024 <br>
    @Last Modified by: Prayag Bhoir<br>
    @Last Modified time: 11-09-2024 <br>
    @Title : Python programs on boto3 crud operations and large file transfer<br>
"""

In [2]:
import boto3
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
s3 = boto3.client('s3')


# 1. CRUD operation using BOTO3

CRUD Operations in S3<br>
	Create: Create buckets, upload files.<br>
	Read: Download files, list files in a bucket.<br>
	Update: Upload a new version of an object.<br>
	Delete: Remove objects or buckets.

### Create S3 bucket and upload a file

In [10]:
def create_bucket(bucket_name, region=None):
    """
    Description:
        Creates an S3 bucket in a specified region. If no region is provided, the bucket is created in the default region.
    Parameters:
        bucket_name (str): The name of the bucket to create.
        region (str): The region where the bucket should be created (optional).
    Return:
        None
    """
    if region is None:
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(Bucket=bucket_name,
                         CreateBucketConfiguration={'LocationConstraint': region})


def main():
	create_bucket('prayag.bhoir', 'us-west-2')
     
if __name__=="__main__":
     main()
    


In [7]:
def upload_file(file_name, bucket_name, object_name=None):
    """
    Description:
        Uploads a file to the specified S3 bucket.
    Parameters:
        file_name (str): The path to the local file to upload.
        bucket_name (str): The name of the S3 bucket.
        object_name (str): The S3 object name (optional). Defaults to file_name.
    Return:
        None
    """
    if object_name is None:
        object_name = file_name
    s3.upload_file(file_name, bucket_name, object_name)


def main():
	upload_file('C:/Users/bhoir/OneDrive/Desktop/pratice_bl/AWS_S3_Operations/crud_files/address_book.json', 'prayag.bhoir', 'address_book.json')

     
if __name__=="__main__":
     main()
    

### Read: Download files, list files in a bucket.

In [15]:
def download_file(bucket_name, object_name, file_name):
    """
    Description:
        Downloads a file from the specified S3 bucket.
    Parameters:
        bucket_name (str): The name of the S3 bucket.
        object_name (str): The name of the S3 object to download.
        file_name (str): The local file path where the downloaded file should be saved.
    Return:
        None
    """
    s3.download_file(bucket_name, object_name, file_name)

def main():
	download_file('prayag.bhoir', 'address_book.json', 'crud_files/downloaded_file.json')
	
     
if __name__=="__main__":
     main()
    

In [16]:
def list_files(bucket_name):
    """
    Description:
        Lists all files (objects) in the specified S3 bucket.
    Parameters:
        bucket_name (str): The name of the S3 bucket.
    Return:
        List of object keys (file names) in the bucket.
    """
    response = s3.list_objects_v2(Bucket=bucket_name)
    
    # Check if the bucket contains any files
    if 'Contents' in response:
        files = [obj['Key'] for obj in response['Contents']]
        return files
    else:
        return []

def main():
	  print(list_files('prayag.bhoir'))
	
     
if __name__=="__main__":
     main()
    

['address_book.json']


### Update: Upload a new version of an object.

In [17]:
import json

In [18]:
def update_file(bucket_name, object_name, update_data):
    """
    Description:
        Updates a JSON file in an S3 bucket by downloading, modifying, and re-uploading it.
    Parameters:
        bucket_name (str): The name of the S3 bucket.
        object_name (str): The key (file name) of the JSON file in the bucket.
        new_data (dict): The new data to merge/update the existing JSON content.
    Return:
        None
    """
    response = s3.get_object(Bucket=bucket_name, Key=object_name)
    
    file_content = response['Body'].read().decode('utf-8')
    json_content = json.loads(file_content)
    
    json_content.update(update_data)
    
    # Convert updated data back to JSON
    updated_content = json.dumps(json_content)
    
    # Upload the updated JSON file back to S3
    s3.put_object(Bucket=bucket_name, Key=object_name, Body=updated_content)
    
    
    
def main():
    update_data = {
        "first_name": "prayag",
        "last_name": "bhoir",
        "address": "1234 MG Road",
        "city": "Panvel",
        "state": "Maharashtra",
        "zip_code": "410206",
        "phone": "8369204930",
        "email": "prayaga.bhoir@elon.com"
    }
    update_file('prayag.bhoir', 'crud_files/address_book.json', update_data)
    
		
if __name__=="__main__":
     main()
    

### Delete: Remove objects or buckets.

In [8]:
def delete_object(bucket_name, object_name):
    """
    Description:
        Deletes an object from the specified S3 bucket.
    Parameters:
        bucket_name (str): The name of the S3 bucket.
        object_name (str): The name of the S3 object to delete.
    Return:
        None
    """
    s3.delete_object(Bucket=bucket_name, Key=object_name)


def main():
	  delete_object('prayag.bhoir', 'address_book.json')
	
     
if __name__=="__main__":
     main()
    

# 2. Transferring Data to/from S3

In [9]:
from boto3.s3.transfer import TransferConfig


In [13]:
def upload_file(file_name, bucket_name, object_name=None):
    """
    Description:
        Uploads a file to the specified S3 bucket.
    Parameters:
        file_name (str): The path to the local file to upload.
        bucket_name (str): The name of the S3 bucket.
        object_name (str): The S3 object name (optional). Defaults to file_name.
    Return:
        None
    """
    
    config = TransferConfig(multipart_threshold=1024*1024*25, max_concurrency=30,
                         multipart_chunksize=1024*1024*25, use_threads=True)
    print("Uploading...")
    
    if object_name is None:
        object_name = file_name
    s3.upload_file(file_name, bucket_name, object_name, Config=config)
    
    print("Done uploading")


def main():
	upload_file('C:/Users/bhoir/OneDrive/Desktop/pratice_bl/AWS_S3_Operations/transfer_files/demo.csv', 'prayag.bhoir', 'name.csv')
	upload_file('C:/Users/bhoir/OneDrive/Desktop/pratice_bl/AWS_S3_Operations/transfer_files/Japan in 8K 60fps.mp4', 'prayag.bhoir', 'big_file.mp4')

     
if __name__=="__main__":
     main()
    

Uploading...
Done uploading
Uploading...
Done uploading


In [ ]:
def download_file(bucket_name, object_name, file_name):
    """
    Description:
        Downloads a file from the specified S3 bucket.
    Parameters:
        bucket_name (str): The name of the S3 bucket.
        object_name (str): The name of the S3 object to download.
        file_name (str): The local file path where the downloaded file should be saved.
    Return:
        None
    """
    s3.download_file(bucket_name, object_name, file_name)

def main():
	download_file('prayag.bhoir', 'name.csv', 'transfer_files/downloaded_file.csv')
	
     
if __name__=="__main__":
     main()
    